In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [246]:
%autoreload
%matplotlib inline

import numpy as np
import pickle

import main


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc_file('matplotlibrc')

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.spectools as spectools

from darkhistory.spec.spectrum import Spectrum

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec

from darkhistory.electrons import positronium

from scipy.interpolate import interp1d


In [117]:
test = positronium.weighted_photon_spec(photeng)
print(test.toteng()/phys.me)

2.000000000081069


# Marching Up Algorithm

In [3]:
ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf = main.load_ics_data()

********* Thomson regime scattered photon spectrum *********
Initializing...
Computing spectra by an expansion in beta...
----> Computation by expansion in beta complete!
Computing spectra by analytic series...
*** Computing series 1/12...
*** Computing series 2/12...
*** Computing series 3/12...
*** Computing series 4/12...
*** Computing series 5/12...
*** Computing series 6/12...
*** Computing series 7/12...
*** Computing series 8/12...
*** Computing series 9/12...
*** Computing series 10/12...
*** Computing series 11/12...
*** Computing series 12/12...
----> Computation by analytic series complete!
Spectrum computed!
********* Relativistic regime scattered photon spectrum *********
Initializing...
Computing series 1/4...
Computing series 2/4...
Computing series 3/4...
Computing series 4/4...
Relativistic Computation Complete!
********* Thomson regime energy loss spectrum *********
Computing nonrelativistic energy loss spectrum...
Computing energy loss spectrum by beta expansion...
C

In [247]:
def get_elec_cooling_tf_fast(
    raw_nonrel_tf, raw_rel_tf, raw_engloss_tf,
    eleceng, photeng, rs, xe, xHe=0
):

    """Returns transfer function for complete electron cooling through ICS and atomic processes.

    Parameters
    ----------
    nonrel_tf : TransFuncAtRedshift
        Raw nonrelativistic primary electron ICS transfer function.
    rel_tf : string
        Raw relativistic primary electron ICS transfer function.
    engloss_tf_filename : string
        Raw primary electron ICS energy loss transfer function.
    eleceng : ndarray
        The electron *kinetic* energy abscissa.
    photeng : ndarray
        The photon energy abscissa.
    rs : float
        The redshift.
    xe : float
        Free electron fraction. 
    xHe : float, optional
        Singly-ionized helium fraction, nHe+/nH. Set to nHe/nH*xe if None.
    
    Returns
    -------

    tuple of TransFuncAtRedshift
        Transfer functions for photons and low energy electrons.

    Note
    ----
    The raw transfer functions should be generated when the code package is first installed. The transfer function corresponds to the fully resolved
    photon spectrum after scattering by one electron.

    This version of the code works faster, but dispenses with energy conservation checks and several other safeguards. Use only with default abscissa, or when get_ics_cooling_tf works.

    """
    
    if xHe is None:
        xHe = xe*phys.nHe/phys.nH
        
    # v/c of electrons, important subsequently.
    beta_ele = np.sqrt(1 - 1/(1 + eleceng/phys.me)**2)
        
    #####################################
    # Inverse Compton
    #####################################

    T = phys.TCMB(rs)

    # Photon transfer function for single primary electron single scattering.
    # This is dN/(dE dt), dt = 1 s.
    phot_ICS_tf = ics_spec(
        eleceng, photeng, T, nonrel_tf = raw_nonrel_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    phot_ICS_tf._grid_vals = phot_ICS_tf.grid_vals.astype('float64')

    # Energy loss transfer function for single primary electron
    # single scattering. This is dN/(dE dt), dt = 1 s.
    engloss_ICS_tf = engloss_spec(
        eleceng, photeng, T, nonrel_tf = raw_engloss_tf, rel_tf = raw_rel_tf
    )

    # Downcasting speeds up np.dot
    engloss_ICS_tf._grid_vals = engloss_ICS_tf.grid_vals.astype('float64')

    # Switch the spectra type here to type 'N'.
    if phot_ICS_tf.spec_type == 'dNdE':
        phot_ICS_tf.switch_spec_type()
    if engloss_ICS_tf.spec_type == 'dNdE':
        engloss_ICS_tf.switch_spec_type()


    # Define some useful lengths.
    N_eleceng = eleceng.size
    N_photeng = photeng.size

    # Create the secondary electron transfer functions.

    # ICS transfer function.
    elec_ICS_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    elec_ICS_tf._grid_vals = spectools.engloss_rebin_fast(
        eleceng, photeng, engloss_ICS_tf.grid_vals, eleceng
    )
    
    # Total upscattered photon energy.
    cont_loss_ICS_vec = np.zeros_like(eleceng)
    # Deposited energy, enforces energy conservation.
    deposited_ICS_vec = np.zeros_like(eleceng)
    
    
    #####################
    # Excitation  
    #####################
    
    # Construct the rate matrices first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    

    rate_matrix_exc_HI = np.diag(
        (1 - xe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_matrix_exc_HeI = np.diag(
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_matrix_exc_HeII = np.diag(
        xHe*phys.nH*rs**3 * phys.coll_exc_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )

    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_exc_HI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HI[:, np.where(eleceng > phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.lya_eng] - phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeI_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeI[:, np.where(eleceng > phys.He_exc_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > phys.He_exc_eng] - phys.He_exc_eng,
        dlnz = -1, spec_type = 'N'
    )
    elec_exc_HeII_tf = tf.TransFuncAtRedshift(
        np.squeeze(rate_matrix_exc_HeII[:, np.where(eleceng > 4*phys.lya_eng)]), 
        in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng[eleceng > 4*phys.lya_eng] - 4*phys.lya_eng,
        dlnz = -1, spec_type = 'N'
    )
    
    # Rebin these transfer functions back to eleceng.
    elec_exc_HI_tf.rebin(eleceng)
    elec_exc_HeI_tf.rebin(eleceng)
    elec_exc_HeII_tf.rebin(eleceng)
   
    # Deposited energy for excitation.
    deposited_exc_vec = np.zeros_like(eleceng)
    
    #####################
    # Ionization  
    #####################
    
    # Construct the rate vector first. Secondary electron spectrum is an electron at in_eng - excitation energy, 
    # with a per second rate given by n*sigma*c.
    rate_vec_ion_HI = (
        (1 - xe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeI = (
        (phys.nHe/phys.nH - xHe)*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeI') * beta_ele * phys.c
    )
    
    rate_vec_ion_HeII = (
        xHe*phys.nH*rs**3 * phys.coll_ion_xsec(eleceng, species='HeII') * beta_ele * phys.c
    )
    
    # Construct the spectra. 
    elec_spec_ion_HI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HI') for rate,in_eng in zip(rate_vec_ion_HI,eleceng)]
    )
    elec_spec_ion_HeI = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeI') for rate,in_eng in zip(rate_vec_ion_HeI,eleceng)]
    )
    elec_spec_ion_HeII = np.array(
        [rate*phys.coll_ion_sec_elec_spec(in_eng, eleceng, species='HeII') for rate,in_eng in zip(rate_vec_ion_HeII,eleceng)]
    )
    
#     for i in np.arange(500):
# #         print(np.dot(elec_spec_ion_HI[i], eleceng) + rate_vec_ion_HI[i]*phys.rydberg - eleceng[i]*rate_vec_ion_HI[i])
#         print('Injection energy: ', eleceng[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng))
#         print(rate_vec_ion_HeI[i]*phys.He_ion_eng)
#         print(eleceng[i]*rate_vec_ion_HeI[i])
#         print(np.dot(elec_spec_ion_HeI[i], eleceng) + rate_vec_ion_HeI[i]*phys.He_ion_eng - eleceng[i]*rate_vec_ion_HeI[i])
    
    
    
    # Construct the TransFuncAtRedshift objects.
    # Electrons scatter from in_eng to in_eng - excitation energy.
    # Remove all of the columns (eng) that have energies below the excitation energy, 
    elec_ion_HI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeI_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeI, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    elec_ion_HeII_tf = tf.TransFuncAtRedshift(
        elec_spec_ion_HeII, in_eng = eleceng, rs = rs*np.ones_like(eleceng), 
        eng = eleceng, dlnz = -1, spec_type = 'N'
    )
    
    # Deposited energy for ionization.
    deposited_ion_vec = np.zeros_like(eleceng)
    
    #############################################
    # Heating
    #############################################
    
    dE_heat_dt = phys.elec_heating_engloss_rate(eleceng, xe, rs)
    
    deposited_heat_vec = np.zeros_like(eleceng)
    
    
    #############################################
    # Initialization of secondary spectra 
    #############################################

    # Low and high energy boundaries
    loweng = 3000
    eleceng_high = eleceng[eleceng > loweng]
    eleceng_high_ind = np.arange(eleceng.size)[eleceng > loweng]
    eleceng_low = eleceng[eleceng <= loweng]
    eleceng_low_ind  = np.arange(eleceng.size)[eleceng <= loweng]


    if eleceng_low.size == 0:
        raise TypeError('Energy abscissa must contain a low energy bin below 3 keV.')

    # Empty containers for quantities.
    # Final secondary photon spectrum.
    sec_phot_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_photeng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = photeng,
        dlnz = -1, spec_type = 'N'
    )
    # Final secondary low energy electron spectrum.
    sec_lowengelec_tf = tf.TransFuncAtRedshift(
        np.zeros((N_eleceng, N_eleceng)), in_eng = eleceng,
        rs = rs*np.ones_like(eleceng), eng = eleceng,
        dlnz = -1, spec_type = 'N'
    )

    # Start building sec_phot_tf and sec_lowengelec_tf.
    # Low energy regime first.

    sec_lowengelec_tf._grid_vals[:eleceng_low.size, :eleceng_low.size] = (
        np.identity(eleceng_low.size)
    )

    # Continuum energy loss rate per electron, dU_CMB/dt.
    CMB_upscatter_eng_rate = phys.thomson_xsec*phys.c*phys.CMB_eng_density(T)
    print('Thomson cross section: ', phys.thomson_xsec)
    print('CMB energy density: ', phys.CMB_eng_density(T))

    # High energy electron loop to get fully resolved spectrum.
    for i, eng in zip(eleceng_high_ind, eleceng_high):
        

        # print('Check energies and indexing: ')
        # print(i, eleceng[i], eng)

        phot_ICS_N = phot_ICS_tf.grid_vals[i]
        
        elec_ICS_N      = elec_ICS_tf.grid_vals[i]
        
        elec_exc_HI_N   = elec_exc_HI_tf.grid_vals[i]
        elec_exc_HeI_N  = elec_exc_HeI_tf.grid_vals[i]
        elec_exc_HeII_N = elec_exc_HeII_tf.grid_vals[i]
        
        elec_ion_HI_N   = elec_ion_HI_tf.grid_vals[i]
        elec_ion_HeI_N  = elec_ion_HeI_tf.grid_vals[i]
        elec_ion_HeII_N = elec_ion_HeII_tf.grid_vals[i]
        
#         if i == 137:
#             utils.compare_arr([eleceng, elec_ion_HI_N])
#             print('Total Number: ', np.sum(elec_ion_HI_N))
#             print('Total Energy: ', np.dot(eleceng, elec_ion_HI_N))
        
        elec_heat_spec = spectools.rebin_N_arr(np.array([1]), np.array([eng]), eleceng)
        elec_heat_spec.eng -= dE_heat_dt[i]
        elec_heat_spec.rebin(eleceng)
        elec_heat_N = elec_heat_spec.N
        
        sec_elec_spec_N = (
            elec_ICS_N 
            + elec_exc_HI_N + elec_exc_HeI_N + elec_exc_HeII_N
            + elec_ion_HI_N + elec_ion_HeI_N + elec_ion_HeII_N
            + elec_heat_N
        )
        sec_phot_spec_N = phot_ICS_N
        

        sec_elec_totN = np.sum(sec_elec_spec_N)
        # The *net* total energy of secondary electrons produced
        # per unit time.
        sec_elec_toteng = np.dot(sec_elec_spec_N, eleceng)
        # The total energy of secondary photons produced per unit time.
        sec_phot_toteng = np.dot(sec_phot_spec_N, photeng)
        # Deposited ICS energy per unit time, dD/dt.
        # Numerical error (should be zero except for numerics)
        print(np.sum(phot_ICS_N))
        print(np.sum(elec_ICS_N))
        deposited_ICS_eng = (
            np.sum(elec_ICS_N)*eng - np.dot(elec_ICS_N, eleceng)
            - (np.dot(phot_ICS_N, photeng) - CMB_upscatter_eng_rate)
        )
        # Deposited excitation energy. 
        deposited_exc_eng = (
            phys.lya_eng*np.sum(elec_exc_HI_N)
            + phys.He_exc_eng*np.sum(elec_exc_HeI_N)
            + 4*phys.lya_eng*np.sum(elec_exc_HeII_N)
        )
        # Deposited ionization energy. Remember that the secondary spectrum
        # has two electrons for each ionization event.
        deposited_ion_eng = (
            phys.rydberg*np.sum(elec_ion_HI_N/2)
            + phys.He_ion_eng*np.sum(elec_ion_HeI_N/2)
            + 4*phys.rydberg*np.sum(elec_ion_HeII_N/2)
        )
        # Deposited heating energy. 
        deposited_heat_eng = dE_heat_dt[i]
        
        # In the original code, the energy of the electron has gamma > 20,
        # then the continuum energy loss is assigned to deposited_eng instead.
        # I'm not sure if this is necessary, but let's be consistent with the
        # original code for now.

        continuum_engloss = CMB_upscatter_eng_rate
        
        if eng + phys.me > 20*phys.me:
            deposited_ICS_eng += CMB_upscatter_eng_rate
            continuum_engloss = 0

        # Remove self-scattering.
        sec_elec_spec_N[i] = 0
        
        # Rescale.
        toteng_no_self_scatter = (
            np.dot(sec_elec_spec_N, eleceng)
            + np.dot(sec_phot_spec_N, photeng)
            - continuum_engloss
            + deposited_ICS_eng
            + deposited_exc_eng
            + deposited_ion_eng
            + deposited_heat_eng
        )
        
        fac = eng/toteng_no_self_scatter
        # Normalize to one electron. 
        
        sec_elec_spec_N    *= fac
        sec_phot_spec_N    *= fac
        continuum_engloss  *= fac
        deposited_ICS_eng  *= fac
        deposited_exc_eng  *= fac
        deposited_ion_eng  *= fac
        deposited_heat_eng *= fac

        # Get the full secondary photon spectrum. Type 'N'
        resolved_phot_spec_vals = np.dot(
            sec_elec_spec_N, sec_phot_tf.grid_vals
        )
        
        # Get the full secondary low energy electron spectrum. Type 'N'.

        resolved_lowengelec_spec_vals = np.dot(
            sec_elec_spec_N, sec_lowengelec_tf.grid_vals
        )

#         utils.compare_arr([eleceng, resolved_lowengelec_spec_vals])
        
        # The resolved lowengelec spectrum is simply one electron
        # in the bin just below 3 keV.
        # Added directly to sec_lowengelec_tf. Removed the dot for speed.
        # resolved_lowengelec_spec_vals = np.zeros_like(eleceng)
        # resolved_lowengelec_spec_vals[eleceng_low_ind[-1]] += 1

        # Add the resolved spectrum to the first scatter.
        sec_phot_spec_N += resolved_phot_spec_vals

        # Resolve the secondary electron continuum loss and deposition.
        continuum_engloss += np.dot(sec_elec_spec_N, cont_loss_ICS_vec)

        deposited_ICS_eng  += np.dot(sec_elec_spec_N, deposited_ICS_vec)
        deposited_exc_eng  += np.dot(sec_elec_spec_N, deposited_exc_vec)
        deposited_ion_eng  += np.dot(sec_elec_spec_N, deposited_ion_vec)
        deposited_heat_eng += np.dot(sec_elec_spec_N, deposited_heat_vec)

        # Now, append the resulting spectrum to the transfer function.
        # Do this without calling append of course: just add to the zeros
        # that fill the current row in _grid_vals.
        sec_phot_tf._grid_vals[i] += sec_phot_spec_N
        sec_lowengelec_tf._grid_vals[i] += resolved_lowengelec_spec_vals
        # Set the correct values in cont_loss_vec and deposited_vec.
        
        cont_loss_ICS_vec[i]  = continuum_engloss
        deposited_ICS_vec[i]  = deposited_ICS_eng
        deposited_exc_vec[i]  = deposited_exc_eng
        deposited_ion_vec[i]  = deposited_ion_eng
        deposited_heat_vec[i] = deposited_heat_eng

        
        check = True
        verbose = True
        failed_conservation_check = False
        
        if check:

            conservation_check = (
                eng
                - np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                + cont_loss_ICS_vec[i]
                - np.dot(sec_phot_tf.grid_vals[i], photeng)
                - deposited_exc_vec[i]
                - deposited_ion_vec[i]
                - deposited_heat_vec[i]
            )

            if conservation_check/eng > 0.1:
                failed_conservation_check = True
                
            if verbose or failed_conservation_check:
                
                print('***************************************************')
                print('rs: ', rs)
                print('injected energy: ', eng)
                print(
                    'Energy in low energy electrons: ',
                    np.dot(sec_lowengelec_tf.grid_vals[i], eleceng)
                )
                print('Energy in photons: ', np.dot(sec_phot_tf.grid_vals[i], photeng))
                print('Continuum_engloss: ', cont_loss_ICS_vec[i])
                print(
                    'Energy in photons - Continuum: ',
                    np.dot(sec_phot_tf.grid_vals[i], photeng) - cont_loss_ICS_vec[i]
                )
                print(
                    'Deposited in ionization: ', deposited_ion_vec[i]
                )
                print(
                    'Deposited in excitation: ', deposited_exc_vec[i]
                )
                print(
                    'Deposited in heating: ', deposited_heat_vec[i]
                )
                print(
                    'Energy is conserved up to (%): ',
                    conservation_check/eng*100
                )
                print('Deposited in ICS: ', deposited_ICS_vec[i])
                print(
                    'Energy conservation with deposited (%): ',
                    (conservation_check - deposited_ICS_vec[i])/eng*100
                )
                print('***************************************************')
                
            if failed_conservation_check:
                raise RuntimeError('Conservation of energy failed.')


    return (sec_phot_tf, sec_lowengelec_tf, cont_loss_ICS_vec)


In [248]:
eleceng = 10**np.arange(0, np.log10(5e12), 0.0254)
photeng = 10**np.arange(-4, np.log10(5e12), 0.0334)


rs = 600
xe = .01

b = get_elec_cooling_tf_fast(
    ics_thomson_ref_tf, ics_rel_ref_tf, engloss_ref_tf,
    eleceng, photeng, rs, xe, xHe=0
)

Thomson cross section:  6.652458734e-25
CMB energy density:  33769934343.76002
0.0017693426250652959
0.00018868430148974896
***************************************************
rs:  600
injected energy:  3018.5613014197993
Energy in low energy electrons:  2901.852478927338
Energy in photons:  2689.608166251477
Continuum_engloss:  2647.716839442689
Energy in photons - Continuum:  41.89132680878811
Deposited in ionization:  40.38844151962926
Deposited in excitation:  28.02771738581055
Deposited in heating:  6.460473298242648
Energy is conserved up to (%):  -0.0019590962085588894
Deposited in ICS:  -0.059136520008484246
Energy conservation with deposited (%):  -2.1762421206016872e-14
***************************************************
0.0017693419375264348
0.00019423000239059208
***************************************************
rs:  600
injected energy:  3200.368594586517
Energy in low energy electrons:  2964.2361521465746
Energy in photons:  5421.598177182115
Continuum_engloss:  5334.52

0.0017693416759580042
0.0004691792880923957
***************************************************
rs:  600
injected energy:  19615.527877010823
Energy in low energy electrons:  6287.741156573007
Energy in photons:  189051.40641678293
Continuum_engloss:  179072.76078540098
Energy in photons - Continuum:  9978.64563138195
Deposited in ionization:  1782.7330813182639
Deposited in excitation:  1301.0319168824497
Deposited in heating:  269.29057586813354
Energy is conserved up to (%):  -0.019956052355711653
Deposited in ICS:  -3.9144850130357276
Energy conservation with deposited (%):  2.560887032311912e-13
***************************************************
0.0017693416907088614
0.00048226564908073936
***************************************************
rs:  600
injected energy:  20796.966871036937
Energy in low energy electrons:  6411.094885840754
Energy in photons:  198577.8905906464
Continuum_engloss:  187655.36751440764
Energy in photons - Continuum:  10922.523076238751
Deposited in ioniz

Deposited in heating:  454.95351178100066
Energy is conserved up to (%):  -0.010743267324348053
Deposited in ICS:  -9.09361121074288
Energy conservation with deposited (%):  -2.1612039448294379e-13
***************************************************
0.0017693416865700094
0.0009126353451869643
***************************************************
rs:  600
injected energy:  89742.87945007472
Energy in low energy electrons:  8913.62282938054
Energy in photons:  502197.9430662003
Continuum_engloss:  427151.6863028586
Energy in photons - Continuum:  75046.25676334172
Deposited in ionization:  3047.3084570231113
Deposited in excitation:  2284.3484501503904
Deposited in heating:  460.6511464291264
Energy is conserved up to (%):  -0.010372072199156326
Deposited in ICS:  -9.308196249981421
Energy conservation with deposited (%):  -2.029740263023259e-13
***************************************************
0.0017693416891805124
0.0009334264472945007
**************************************************

0.0017693417441916573
0.001441870050627527
***************************************************
rs:  600
injected energy:  410582.0889699127
Energy in low energy electrons:  10076.61041832433
Energy in photons:  1055491.0205046886
Continuum_engloss:  661878.0807721843
Energy in photons - Continuum:  393612.9397325043
Deposited in ionization:  3602.4479932947684
Deposited in excitation:  2735.4922747113337
Deposited in heating:  569.0336110392221
Energy is conserved up to (%):  -0.003515754912096441
Deposited in ICS:  -14.4350599609173
Energy conservation with deposited (%):  -5.616059121998499e-14
***************************************************
0.0017693417478013454
0.0014586269516349403
***************************************************
rs:  600
injected energy:  435311.3592296377
Energy in low energy electrons:  10098.071580345135
Energy in photons:  1087849.9895694044
Continuum_engloss:  669550.7040218585
Energy in photons - Continuum:  418299.2855475459
Deposited in ionization:

0.0017693418138737512
0.0017150503787735462
***************************************************
rs:  600
injected energy:  1878451.5586741285
Energy in low energy electrons:  10343.92276991235
Energy in photons:  2673150.997317826
Continuum_engloss:  812219.9127360125
Energy in photons - Continuum:  1860931.0845818133
Deposited in ionization:  3729.8427193314114
Deposited in excitation:  2842.6858277463098
Deposited in heating:  621.7405143555271
Energy is conserved up to (%):  -0.0009432097915187052
Deposited in ICS:  -17.717739028972865
Energy conservation with deposited (%):  -7.331847785967228e-14
***************************************************
0.001769341814308625
0.001719571102348489
***************************************************
rs:  600
injected energy:  1991590.2890576117
Energy in low energy electrons:  10346.84451816546
Energy in photons:  2789982.0592415873
Continuum_engloss:  815918.9058954995
Energy in photons - Continuum:  1974063.1533460878
Deposited in ionizat

...

0.00175987780671824
0.001759877806718239
***************************************************
rs:  600
injected energy:  1393156802.9453034
Energy in low energy electrons:  10356.452839285823
Energy in photons:  1393974605.7696404
Continuum_engloss:  871481.6099347362
Energy in photons - Continuum:  1393103124.1597056
Deposited in ionization:  3737.965411878258
Deposited in excitation:  2850.434655848771
Deposited in heating:  647.190609625588
Energy is conserved up to (%):  0.002590285745637696
Deposited in ICS:  36086.74208030543
Energy conservation with deposited (%):  5.5170933294956994e-14
***************************************************
0.0017593122353662966
0.0017593122353662953
***************************************************
rs:  600
injected energy:  1477066335.3378193
Energy in low energy electrons:  10356.452743449077
Energy in photons:  1477884122.80898
Continuum_engloss:  871481.6009127994
Energy in photons - Continuum:  1477012641.2080672
Deposited in ionization:  3

0.0017252326583055303
0.0017252326583055313
***************************************************
rs:  600
injected energy:  6757716993.552843
Energy in low energy electrons:  10356.452000258641
Energy in photons:  6758534578.1149845
Continuum_engloss:  871481.530783117
Energy in photons - Continuum:  6757663096.584202
Deposited in ionization:  3737.9651211879786
Deposited in excitation:  2850.4344429800817
Deposited in heating:  647.2330501630038
Energy is conserved up to (%):  0.000537235934280298
Deposited in ICS:  36304.88402791577
Energy conservation with deposited (%):  -2.3435206532236685e-14
***************************************************
0.001722716956760043
0.001722716956760045
***************************************************
rs:  600
injected energy:  7164732823.911086
Energy in low energy electrons:  10356.451995942822
Energy in photons:  7165550405.088212
Continuum_engloss:  871481.5303734717
Energy in photons - Continuum:  7164678923.557838
Deposited in ionization:  

***************************************************
rs:  600
injected energy:  43913696902.44064
Energy in low energy electrons:  10356.45196068745
Energy in photons:  43914514432.8191
Continuum_engloss:  871481.5270182687
Energy in photons - Continuum:  43913642951.29208
Deposited in ionization:  3737.9651075234533
Deposited in excitation:  2850.4344330385043
Deposited in heating:  647.2440207564614
Energy is conserved up to (%):  8.279661154770255e-05
Deposited in ICS:  36359.0530439003
Energy conservation with deposited (%):  -7.630960504431856e-15
***************************************************
0.0015308810345757732
0.0015308810345757737
***************************************************
rs:  600
injected energy:  46558609352.295815
Energy in low energy electrons:  10356.451960560862
Energy in photons:  46559426881.98198
Continuum_engloss:  871481.5270061303
Energy in photons - Continuum:  46558555400.45497
Deposited in ionization:  3737.965107479875
Deposited in excitation:  

0.0011314499066111946
0.0011314499066111944
***************************************************
rs:  600
injected energy:  225839550114.55496
Energy in low energy electrons:  10356.451959471136
Energy in photons:  225840367632.64142
Continuum_engloss:  871481.5269013259
Energy in photons - Continuum:  225839496151.1145
Deposited in ionization:  3737.965107105182
Deposited in excitation:  2850.434432736689
Deposited in heating:  647.2466222158909
Energy is conserved up to (%):  1.6104948098934075e-05
Deposited in ICS:  36371.34226014858
Energy conservation with deposited (%):  3.217625335420095e-14
***************************************************
0.0011135414109841267
0.0011135414109841267
***************************************************
rs:  600
injected energy:  239441817286.33752
Energy in low energy electrons:  10356.451959463726
Energy in photons:  239442634804.14844
Continuum_engloss:  871481.5269006077
Energy in photons - Continuum:  239441763322.62155
Deposited in ionizati

0.0006202739292239942
0.0006202739292239939
***************************************************
rs:  600
injected energy:  1231402412206.2134
Energy in low energy electrons:  10356.451959396287
Energy in photons:  1231403229717.6265
Continuum_engloss:  871481.5268940409
Energy in photons - Continuum:  1231402358236.0996
Deposited in ionization:  3737.9651070795694
Deposited in excitation:  2850.4344327183885
Deposited in heating:  647.2480983265498
Energy is conserved up to (%):  2.954193841632489e-06
Deposited in ICS:  36378.01401222066
Energy conservation with deposited (%):  1.7450771101704808e-14
***************************************************
0.0006047373544382455
0.0006047373544382456
***************************************************
rs:  600
injected energy:  1305569512691.0994
Energy in low energy electrons:  10356.451959395634
Energy in photons:  1305570330202.2969
Continuum_engloss:  871481.5268939771
Energy in photons - Continuum:  1305569458720.77
Deposited in ionizat

In [245]:
4*phys.stefboltz/phys.c*phys.kB**4

0.004722157511970381

In [249]:
3018.5613 - 2901.8524789 - 41.8913268

74.81749430000002

In [212]:
np.set_printoptions(threshold=np.nan)
a = phys.coll_ion_sec_elec_spec(3018.56, eleceng, species='HeI')
print(np.sum(a))
print(np.dot(a, eleceng) + phys.He_ion_eng)
utils.compare_arr([eleceng, a])

2.0
3018.5599999999995
[[1.00000000e+00 2.57549312e-03]
 [1.06022978e+00 2.72953357e-03]
 [1.12408719e+00 2.89263807e-03]
 [1.19179072e+00 3.06531043e-03]
 [1.26357202e+00 3.24807657e-03]
 [1.33967669e+00 3.44148429e-03]
 [1.42036512e+00 3.64610263e-03]
 [1.50591341e+00 3.86252099e-03]
 [1.59661425e+00 4.09134764e-03]
 [1.69277798e+00 4.33320773e-03]
 [1.79473363e+00 4.58874062e-03]
 [1.90283004e+00 4.85859632e-03]
 [2.01743709e+00 5.14343100e-03]
 [2.13894688e+00 5.44390130e-03]
 [2.26777519e+00 5.76065731e-03]
 [2.40436280e+00 6.09433392e-03]
 [2.54917705e+00 6.44554033e-03]
 [2.70271343e+00 6.81484744e-03]
 [2.86549728e+00 7.20277277e-03]
 [3.03808556e+00 7.60976277e-03]
 [3.22106879e+00 8.03617193e-03]
 [3.41507307e+00 8.48223872e-03]
 [3.62076218e+00 8.94805783e-03]
 [3.83883990e+00 9.43354872e-03]
 [4.07005239e+00 9.93842027e-03]
 [4.31519077e+00 1.04621318e-02]
 [4.57509377e+00 1.10038503e-02]
 [4.85065068e+00 1.15624054e-02]
 [5.14280432e+00 1.21362418e-02]
 [5.45255431e+00 1.2

0